In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import MinMaxScaler, StandardScaler #MinMax serve a normalizzare x e y tra 0 e 1 per il clustering
from collections import Counter
from scipy import stats
%matplotlib inline

df_test = pd.read_csv("C:/Users/Mario/Desktop/UniPi/1° semestre/Data Mining 1/Datasets/DatasetProject1/Test_HR_Employee_Attrition.csv")
df_train = pd.read_csv("C:/Users/Mario/Desktop/UniPi/1° semestre/Data Mining 1/Datasets/DatasetProject1/Train_HR_Employee_Attrition.csv")

frame = [df_test, df_train]
df3 = pd.concat(frame)
df3.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,27.0,Yes,Travel_Frequently,763,Sales,15,2,Medical,1,Male,...,4,80.0,2,7,1.0,2,2.0,2,2,2
1,30.0,Yes,Travel_Rarely,1079,Sales,16,4,Marketing,1,Male,...,1,NaN,3,10,2.0,3,8.0,0,0,0
2,56.0,No,Non-Travel,150,Research & Development,2,4,Technical Degree,4,Male,...,3,80.0,1,12,2.0,2,13.0,3,1,4
3,41.0,Yes,NaN,359,Human Resources,18,5,Human Resources,4,Male,...,3,80.0,0,2,4.0,3,3.0,0,0,0
4,42.0,No,NaN,642,Research & Development,1,3,Life Sciences,4,Male,...,1,NaN,1,12,6.0,3,5.0,3,1,0


**DATA UNDERSTANDING**

In [2]:
#HO AGGIUNTO PER PROVA "MonthlyIncome", "YearsWithCurrManager", "YearsSinceLastPromotion", "TrainingTimesLastYear", "NumCompaniesWorked", "MaritalStatus", "PerformanceRating"
       
new_df = df3[['Age', 'Attrition', 'BusinessTravel',  'Department', 'DistanceFromHome', 'Education', 'EducationField','HourlyRate',
       'EnvironmentSatisfaction','Gender',  'JobLevel',  'JobSatisfaction', 'OverTime', 'PercentSalaryHike', 'WorkLifeBalance',
        "MonthlyIncome", "TrainingTimesLastYear", "NumCompaniesWorked", "MaritalStatus", "PerformanceRating", 
        'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole', "YearsWithCurrManager",]]

#new_df.head()

In [3]:
#Riempio i valori mancanti
valori = {"Age": new_df["Age"].mean(),
          "Gender": "Male", 
          "BusinessTravel": "TravelRarely", 
          "YearsAtCompany": new_df["YearsAtCompany"].median(),
          "TrainingTimesLastYear": new_df["TrainingTimesLastYear"].median(),
          "PerformanceRating": new_df["PerformanceRating"].median()}

new_df = new_df.fillna(value=valori)

#Separo new_df in due dataset, uno filtrato per Male e uno per Female 
male_df = new_df[new_df["Gender"] == "Male"]
female_df = new_df[new_df["Gender"] == "Female"]

#Sostituisco i valori vuoti di MonthlyIncome con la mediana per Male
male_income = {"MonthlyIncome": 4865.5}
male_df = male_df.fillna(value=male_income)

#Sostituisco i valori vuoti di MonthlyIncome con la mediana per Female
female_income = {"MonthlyIncome": 5197.0}
female_df = female_df.fillna(value=female_income)

#Riunisco i dataset che avevo separato, con tutti i missing values riempiti
frame = [male_df, female_df]
new_df = pd.concat(frame)

#df contiene il dataset con i missing values riempiti
df = new_df
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,HourlyRate,EnvironmentSatisfaction,Gender,...,WorkLifeBalance,MonthlyIncome,TrainingTimesLastYear,NumCompaniesWorked,MaritalStatus,PerformanceRating,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole,YearsWithCurrManager
0,27.0,Yes,Travel_Frequently,Sales,15,2,Medical,59,1,Male,...,2,4298.0,1.0,6,Divorced,3.0,7,2.0,2,2
1,30.0,Yes,Travel_Rarely,Sales,16,4,Marketing,70,1,Male,...,3,5304.0,2.0,2,Married,4.0,10,8.0,0,0
2,56.0,No,Non-Travel,Research & Development,2,4,Technical Degree,60,4,Male,...,2,6306.0,2.0,2,Divorced,4.0,12,13.0,3,4
3,41.0,Yes,TravelRarely,Human Resources,18,5,Human Resources,89,4,Male,...,3,6430.0,4.0,0,Married,3.0,2,3.0,0,0
4,42.0,No,TravelRarely,Research & Development,1,3,Life Sciences,76,4,Male,...,3,2766.0,6.0,3,Married,4.0,12,5.0,3,0


In [4]:
TotalWorkingYears_UL = df.TotalWorkingYears.mean() + 3*df.TotalWorkingYears.std()
TotalWorkingYears_LL = df.TotalWorkingYears.mean() - 3*df.TotalWorkingYears.std()

print(TotalWorkingYears_UL)
print(TotalWorkingYears_LL)

df[df.TotalWorkingYears > TotalWorkingYears_UL]

34.62193686327969
-12.062753189810303


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,HourlyRate,EnvironmentSatisfaction,Gender,...,WorkLifeBalance,MonthlyIncome,TrainingTimesLastYear,NumCompaniesWorked,MaritalStatus,PerformanceRating,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole,YearsWithCurrManager
39,27.000000,No,Travel_Rarely,Sales,10,4,Medical,37,4,Male,...,2,3041.0,3.0,0,Single,3.0,38,4.0,10,8
265,55.000000,No,Travel_Rarely,Human Resources,26,4,Human Resources,71,3,Male,...,3,19038.0,3.0,4,Married,3.0,35,1.0,9,4
29,40.000000,No,TravelRarely,Research & Development,1,3,Medical,81,4,Male,...,3,10475.0,2.0,0,Single,3.0,37,18.0,10,13
101,37.000000,Yes,Travel_Rarely,Research & Development,2,4,Life Sciences,31,4,Male,...,3,4197.0,2.0,7,Single,3.0,40,1.0,15,8
173,34.000000,No,Travel_Rarely,Sales,7,2,Marketing,78,1,Male,...,3,2932.0,3.0,7,Married,3.0,35,5.0,2,4
181,23.000000,No,Travel_Rarely,Sales,18,5,Life Sciences,83,1,Male,...,3,1223.0,2.0,3,Single,4.0,37,1.0,9,7
323,36.000000,No,Travel_Frequently,Human Resources,8,4,Life Sciences,99,4,Male,...,3,5562.0,3.0,6,Single,3.0,36,3.0,3,7
446,37.115262,No,Travel_Rarely,Research & Development,14,4,Medical,61,3,Male,...,3,18880.0,3.0,8,Single,3.0,36,22.0,15,15
477,30.000000,No,Travel_Rarely,Sales,26,5,Marketing,60,3,Male,...,3,3833.0,2.0,1,Married,4.0,36,2.0,6,13
676,30.000000,No,Travel_Rarely,Research & Development,7,3,Life Sciences,49,4,Male,...,2,7140.0,2.0,4,Single,3.0,37,7.0,4,2


In [5]:
YearsAtCompany_UL = df.YearsAtCompany.mean() + 3*df.YearsAtCompany.std()
YearsAtCompany_LL = df.YearsAtCompany.mean() - 3*df.YearsAtCompany.std()

print(YearsAtCompany_UL)
print(YearsAtCompany_LL)

df[df.YearsAtCompany > YearsAtCompany_UL]

24.52944430479668
-10.839648386429335


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,HourlyRate,EnvironmentSatisfaction,Gender,...,WorkLifeBalance,MonthlyIncome,TrainingTimesLastYear,NumCompaniesWorked,MaritalStatus,PerformanceRating,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole,YearsWithCurrManager
25,51.000000,No,Travel_Frequently,Research & Development,1,3,Life Sciences,90,4,Male,...,4,19847.0,5.0,7,Married,3.0,19,29.0,11,9
41,55.000000,No,Travel_Rarely,Research & Development,2,4,Medical,62,2,Male,...,2,19045.0,3.0,2,Married,3.0,29,36.0,1,7
43,37.115262,Yes,Travel_Rarely,Sales,4,3,Technical Degree,86,4,Male,...,3,14336.0,3.0,0,Single,3.0,9,25.0,7,7
12,52.000000,No,Travel_Rarely,Research & Development,9,3,Life Sciences,60,2,Male,...,1,4865.5,3.0,8,Divorced,3.0,20,34.0,9,15
38,50.000000,No,Non-Travel,Research & Development,10,3,Life Sciences,90,4,Male,...,3,17924.0,3.0,7,Married,3.0,15,31.0,8,7
57,58.000000,No,Travel_Rarely,Research & Development,11,3,Life Sciences,95,2,Male,...,3,4865.5,2.0,1,Single,3.0,6,31.0,5,3
205,49.000000,No,Travel_Rarely,Research & Development,19,3,Technical Degree,34,4,Male,...,4,19502.0,5.0,4,Divorced,3.0,8,31.0,2,2
598,50.000000,No,Travel_Frequently,Sales,14,2,Life Sciences,84,4,Male,...,3,18200.0,2.0,1,Married,3.0,5,32.0,4,2
856,44.000000,No,Travel_Rarely,Research & Development,3,3,Life Sciences,87,1,Male,...,2,19190.0,4.0,1,Single,3.0,2,25.0,2,2
921,58.000000,No,Travel_Rarely,Sales,14,2,Life Sciences,87,3,Male,...,2,13872.0,1.0,4,Married,3.0,18,37.0,7,7


In [6]:
YearsInCurrentRole_UL = df.YearsInCurrentRole.mean() + 3*df.YearsInCurrentRole.std()
YearsInCurrentRole_LL = df.YearsInCurrentRole.mean() - 3*df.YearsInCurrentRole.std()

print(YearsInCurrentRole_UL)
print(YearsInCurrentRole_LL)

df[df.YearsInCurrentRole > YearsInCurrentRole_UL]

15.09866280469219
-6.640159403331647


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,HourlyRate,EnvironmentSatisfaction,Gender,...,WorkLifeBalance,MonthlyIncome,TrainingTimesLastYear,NumCompaniesWorked,MaritalStatus,PerformanceRating,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole,YearsWithCurrManager
42,47.000000,No,Travel_Frequently,Research & Development,2,4,Medical,82,1,Male,...,4,4865.5,3.0,3,Married,3.0,26,5.0,17,6
288,30.000000,No,TravelRarely,Sales,3,3,Technical Degree,45,1,Male,...,3,3072.0,4.0,2,Divorced,3.0,25,12.0,17,8
75,30.000000,No,Travel_Rarely,Research & Development,4,2,Technical Degree,58,3,Male,...,3,4152.0,3.0,1,Married,3.0,22,11.0,17,15
341,37.115262,No,Travel_Rarely,Sales,2,5,Life Sciences,91,2,Male,...,3,2632.0,3.0,7,Married,3.0,22,5.0,16,8
450,38.000000,No,Travel_Rarely,Sales,25,3,Life Sciences,99,1,Male,...,2,4771.0,0.0,7,Single,3.0,28,5.0,16,9
582,26.000000,No,Travel_Rarely,Sales,1,1,Life Sciences,99,2,Male,...,3,3578.0,2.0,1,Married,3.0,20,7.0,16,6
762,49.000000,No,Travel_Rarely,Research & Development,10,3,Medical,99,2,Male,...,3,10466.0,3.0,3,Married,3.0,20,8.0,16,11
1025,40.000000,No,Travel_Rarely,Research & Development,7,1,Life Sciences,55,2,Male,...,3,2741.0,3.0,1,Divorced,3.0,21,7.0,16,10
6,40.000000,No,Travel_Frequently,Research & Development,6,3,Life Sciences,51,1,Female,...,3,5605.0,2.0,7,Single,3.0,23,20.0,18,15
113,54.000000,No,Travel_Rarely,Research & Development,9,3,Medical,64,1,Female,...,4,17426.0,6.0,2,Divorced,4.0,21,10.0,16,11


In [7]:
YearsWithCurrManager_UL = df.YearsWithCurrManager.mean() + 3*df.YearsWithCurrManager.std()
YearsWithCurrManager_LL = df.YearsWithCurrManager.mean() - 3*df.YearsWithCurrManager.std()

print(YearsWithCurrManager_UL)
print(YearsWithCurrManager_LL)

df[df.YearsWithCurrManager > YearsWithCurrManager_UL]

14.827537613322004
-6.581279109920644


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,HourlyRate,EnvironmentSatisfaction,Gender,...,WorkLifeBalance,MonthlyIncome,TrainingTimesLastYear,NumCompaniesWorked,MaritalStatus,PerformanceRating,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole,YearsWithCurrManager
54,48.000000,No,Travel_Rarely,Research & Development,29,4,Other,32,4,Male,...,3,10999.0,3.0,1,Single,3.0,20,15.0,11,17
178,37.115262,No,Travel_Frequently,Research & Development,7,4,Medical,42,1,Male,...,3,4284.0,2.0,3,Married,4.0,24,4.0,6,17
12,52.000000,No,Travel_Rarely,Research & Development,9,3,Life Sciences,60,2,Male,...,1,4865.5,3.0,8,Divorced,3.0,20,34.0,9,15
75,30.000000,No,Travel_Rarely,Research & Development,4,2,Technical Degree,58,3,Male,...,3,4152.0,3.0,1,Married,3.0,22,11.0,17,15
81,35.000000,No,Travel_Rarely,Research & Development,28,3,Life Sciences,32,4,Male,...,4,4865.5,5.0,3,Married,3.0,24,5.0,6,17
146,29.000000,No,Travel_Frequently,Research & Development,9,3,Life Sciences,66,4,Male,...,3,4865.5,0.0,1,Married,3.0,17,6.0,14,15
446,37.115262,No,Travel_Rarely,Research & Development,14,4,Medical,61,3,Male,...,3,18880.0,3.0,8,Single,3.0,36,22.0,15,15
618,29.000000,No,Travel_Frequently,Sales,26,4,Marketing,66,1,Male,...,2,4554.0,3.0,2,Married,3.0,29,10.0,6,17
941,30.000000,No,Travel_Rarely,Research & Development,14,3,Life Sciences,95,4,Male,...,2,2862.0,2.0,1,Divorced,3.0,18,10.0,7,17
1159,36.000000,No,Travel_Rarely,Research & Development,1,4,Life Sciences,98,1,Male,...,3,2143.0,3.0,2,Divorced,3.0,22,5.0,7,16
